In [ ]:
flex_positions = ["RB", "WR", "TE"]

# Constraint für Flex-Position
prob += pulp.lpSum([player_vars[i] for i in test if df_optimization.loc[i, 'position_group'] in flex_positions]) >= position_requirements["FLEX"]

In [ ]:
def optimize_team(all_players, df_optimization):
    position_requirements = {
        'QB': 1,
        'RB': 2,
        'WR': 2,
        'TE': 1,
    }
    total_players = 6

    # Initialisiere das Problem
    prob = pulp.LpProblem("FantasyFootballProblem", pulp.LpMaximize)

    # Erstelle eine Liste der Spieler (in diesem Fall verwende ich die player_id)
    players = all_players

    # Erstelle ein Wörterbuch mit den Vorhersagen
    predicted_points = df_optimization.loc[players, 'predicted_points'].to_dict()

    # Erstelle ein Wörterbuch mit den Positionen
    positions = df_optimization.loc[players, 'position_group'].to_dict()

    # Erstelle die Variablen für das Problem
    player_vars = pulp.LpVariable.dicts("Player", [player.replace('Player_', '') for player in players], 0, 1, cat='Integer')

    # Zielfunktion: Maximiere die Gesamtpunktzahl
    prob += pulp.lpSum([player_vars[i] * predicted_points[i] for i in players])

    # Constraints für Positionen
    for pos in set(positions.values()):
        prob += pulp.lpSum([player_vars[i] for i in players if positions[i] == pos]) == position_requirements[pos]

    # Constraints für Gesamtzahl der ausgewählten Spieler
    prob += pulp.lpSum([player_vars[i] for i in players]) == total_players

    # Löse das Problem
    prob.solve()

    # Erstelle eine Liste der ausgewählten Spieler
    chosen_players = [v.name.replace('Player_', '') for v in prob.variables() if v.varValue == 1]

    # Gib die Liste der ausgewählten Spieler zurück
    return chosen_players

In [ ]:
chosen_players = []

all_players = df_optimization.index.tolist()

position_counts = {pos: 0 for pos in position_requirements.keys()}

total_rounds = 60
def pick_player(player_list, position_counts):
    for player in player_list:
        if position_counts[df_optimization.loc[player, 'position_group']] < position_requirements[df_optimization.loc[player, 'position_group']]:
            return player
    return None

for round in range(total_rounds):
    # Führen Sie die Optimierung für die verfügbaren Spieler durch
    test = [player for player in all_players if player not in chosen_players]
    best_players = optimize_team(test, df_optimization)  # Ihre Optimierungsfunktion hier
    
    # Wählen Sie Ihren Spieler
    my_player = best_players[0]  # Nehmen wir an, wir wählen immer den besten Spieler


    
    chosen_players.append(my_player)
    position_counts[df_optimization.loc[my_player, 'position_group']] += 1
    
    # Entfernen Sie diesen Spieler aus der Liste der verfügbaren Spieler
    all_players.remove(my_player)
    
    # Wiederholen Sie den Vorgang für die anderen Spieler
    for other_player in range(num_other_players):
        their_player = pick_player(best_players, position_counts)
        if their_player is not None:
            chosen_players.append(their_player)
            position_counts[df_optimization.loc[their_player, 'position_group']] += 1
            all_players.remove(their_player)


In [ ]:
# Vor dem Optimierungsproblem:
opponent_picks = {}  # Dieses Dictionary wird die Picks jedes Managers in jeder Woche speichern
weeks = all_weeks
position = player_data['position_group'].unique().tolist()
for manager in managers:
    if manager != 'Manager1':  # Wir betrachten nur die gegnerischen Manager
        for week in weeks:
            # Wählen Sie die Spieler mit den höchsten erwarteten Punkten in jeder Position aus
            qb_pick = max([(i, predicted_points_per_strength[(i, week)]) for i in players if position[i] == 'QB' and (i, week) in predicted_points_per_strength], key=lambda x: x[1])[0]
            wr_picks = sorted([(i, predicted_points_per_strength[(i, week)]) for i in players if position[i] == 'WR' and (i, week) in predicted_points_per_strength], key=lambda x: x[1], reverse=True)[:2]  # Die besten zwei WRs
            rb_picks = sorted([(i, predicted_points_per_strength[(i, week)]) for i in players if position[i] == 'RB' and (i, week) in predicted_points_per_strength], key=lambda x: x[1], reverse=True)[:2]  # Die besten zwei RBs
            te_pick = max([(i, predicted_points_per_strength[(i, week)]) for i in players if position[i] == 'TE' and (i, week) in predicted_points_per_strength], key=lambda x: x[1])[0]
            
            # Speichern Sie die Picks
            opponent_picks[(manager, week)] = [qb_pick] + [pick[0] for pick in wr_picks] + [pick[0] for pick in rb_picks] + [te_pick]

# Berechnen Sie die erforderlichen Punkte, um den gegnerischen Manager zu schlagen
required_points_to_beat_opponent = {}
predicted_points_per_strength = predicted_points_per_strength.to_dict()

for week in weeks:
    required_points_to_beat_opponent[week] = sum(predicted_points_per_strength[(i, week)] for manager in managers if manager != 'Manager1' for i in opponent_picks[(manager, week)])

In [ ]:
# Erstelle ein DataFrame nur mit den Features, die wir verwenden wollen
features = [ 'recent_team', 'opponent', 'avg_fantasy_points_allowed', 'is_home', 'passing_air_yards', 'passing_yards_after_catch', 'rushing_yards']
X = player_data[features]
y = player_data['fantasy_points_ppr']

# Da wir kategorische Features haben, müssen wir sie in numerische Werte umwandeln
X = pd.get_dummies(X)

# Teile die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Erstelle das lineare Regressionsmodell
model = LinearRegression()

# Trainiere das Modell mit den Trainingsdaten
model.fit(X_train, y_train)

# Mache Vorhersagen mit den Testdaten
predictions = model.predict(X_test)

# Bewerte das Modell
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

In [ ]:

# Erstelle das XGBoost Regressionsmodell
model = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Trainiere das Modell mit den Trainingsdaten
model.fit(X_train, y_train, 
    early_stopping_rounds=5, 
    eval_set=[(X_test, y_test)], 
    verbose=False)

# Mache Vorhersagen mit den Testdaten
predictions = model.predict(X_test)

# Bewerte das Modell
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

In [ ]:
# Erstelle ein DataFrame nur mit den Features, die wir verwenden wollen
players_features = player_data[features]

# Da wir kategorische Features haben, müssen wir sie in numerische Werte umwandeln
players_features = pd.get_dummies(players_features)

# Beachten Sie, dass wir nach dem Anwenden von get_dummies() sicherstellen müssen, dass die gleichen Spalten im 'players_features'- und 'X_train'-Dataframe vorhanden sind.
missing_cols = set(X_train.columns) - set(players_features.columns)
for c in missing_cols:
    players_features[c] = 0
players_features = players_features[X_train.columns]

# Mache Vorhersagen mit den Spielerdaten
player_predictions = model.predict(players_features)

# Füge die Vorhersagen zum 'players'-DataFrame hinzu
player_data['predicted_fantasy_points'] = player_predictions

print(player_data)

In [ ]:
# Erstelle ein DataFrame nur mit den Features, die wir verwenden wollen
features = [ 'recent_team', 'opponent', 'avg_fantasy_points_allowed', 'is_home', 'passing_air_yards', 'passing_yards_after_catch', 'rushing_yards']
player_data = player_data.dropna()
X = player_data[features]
y = player_data['fantasy_points_ppr']

# Da wir kategorische Features haben, müssen wir sie in numerische Werte umwandeln
X = pd.get_dummies(X)

# Teile die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Erstelle das lineare Regressionsmodell
model = LinearRegression()

# Trainiere das Modell mit den Trainingsdaten
model.fit(X_train, y_train)

# Mache Vorhersagen mit den Testdaten
predictions = model.predict(X_test)

# Bewerte das Modell
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

In [ ]:
# Erstelle ein DataFrame nur mit den Features, die wir verwenden wollen
players_features = player_data[features]

# Da wir kategorische Features haben, müssen wir sie in numerische Werte umwandeln
players_features = pd.get_dummies(players_features)

# Beachten Sie, dass wir nach dem Anwenden von get_dummies() sicherstellen müssen, dass die gleichen Spalten im 'players_features'- und 'X_train'-Dataframe vorhanden sind.
missing_cols = set(X_train.columns) - set(players_features.columns)
for c in missing_cols:
    players_features[c] = 0
players_features = players_features[X_train.columns]

# Mache Vorhersagen mit den Spielerdaten
player_predictions = model.predict(players_features)

# Füge die Vorhersagen zum 'players'-DataFrame hinzu
player_data['predicted_fantasy_points'] = player_predictions

print(player_data)

In [ ]:
# Spieplan Manager
def create_round_robin(players):
    n = len(players)
    schedule = []

    for round in range(n-1):
        for i in range(n//2):
            home = players[i]
            away = players[n - i - 1 - round]
            if i % 2 == round % 2:
                home, away = away, home
            schedule.append((round + 1, home, away))

    return schedule

def add_extra_games(schedule, players):
    n = len(players)
    rounds = len(schedule) // (n // 2)
    
    for round in range(rounds, rounds + (n - 1)//2):
        for i in range(n//2):
            home = players[i]
            away = players[n - i - 1]
            if i % 2 == round % 2:
                home, away = away, home
            schedule.append((round + 1, home, away))
    
    return schedule

managers = ['Manager_' + str(i) for i in range(1, 11)]

round_robin_schedule = create_round_robin(managers)
full_schedule = add_extra_games(round_robin_schedule, managers)

manager_schedule = pd.DataFrame(full_schedule, columns=['week', 'Manager', 'Opponent'])
manager_schedule.sort_values(by=['week', 'Manager'], inplace=True)

In [ ]:
from gurobipy import *

# Liste der Spieler und Teams erstellen
players = player_data['player_id'].unique().tolist()
teams = manager_schedule['Manager'].unique().tolist()
weeks = list(range(1, 14)) 
position = dict(zip(player_data['player_id'], player_data['position_group']))
# Angenommen, jedes Team wählt 15 Spieler
num_players_per_team = 10
rounds = list(range(1, num_players_per_team+1))

# Erstelle ein dictionary, das den Rang für jeden Spieler über die gesamte Saison speichert
season_ranking = {row['player_id']: row['average_rank'] for index, row in players_rank.iterrows()}

# Initialisieren Sie das Modell
m = Model("Fantasy_Football_Optimization")

# x[i][j][t] ist 1, wenn Spieler i in Woche j von Team t ausgewählt und aufgestellt wird
x = m.addVars(players, weeks, teams, vtype=GRB.BINARY, name="chosen")

# y[i][t] ist 1, wenn Spieler i von Team t für die gesamte Saison ausgewählt wird
y = m.addVars(players, teams, vtype=GRB.BINARY, name="selected_by")

# z[t][r] ist 1, wenn Team t in Runde r einen Spieler auswählt
z = m.addVars(teams, rounds, vtype=GRB.BINARY, name="drafted")

# Zielfunktion: Maximiere die Gesamtpunktzahl für Manager_1 über alle Wochen
m.setObjective(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players for j in weeks), GRB.MAXIMIZE)

# Spielplan-Nebenbedingungen
for j in weeks:
    opponent = schedule['Manager_1'][j]
    m.addConstr(quicksum(x[i, j, 'Manager_1'] * expected_points_dict[(i, j)] for i in players) >= quicksum(x[i, j, opponent] * expected_points_dict[(i, j)] for i in players))

for t in teams:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for i in players) == 9)
# Nebenbedingungen hinzufügen
# Ändern Sie die Einschränkungen für die Anzahl der Spieler pro Position
for t in teams:
    for j in weeks:
        # Es sollte einen QB geben
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'QB') == 1)
        # Es sollten mindestens 2 WRs und insgesamt 3 Spieler aus WR, RB oder TE (ohne den speziellen TE) geben
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] in ['WR', 'RB']) + 
                    quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 3)
        # Es sollte mindestens 2 RBs geben
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'RB') >= 2)
        # Es sollte mindestens einen TE geben
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'TE') >= 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'K') == 1)
        m.addConstr(quicksum(x[i, j, t] for i in players if position[i] == 'DST') == 2)



# Ein Spieler kann nur in einer Woche von einem Team ausgewählt werden
for i in players:
    for j in weeks:
        m.addConstr(quicksum(x[i, j, t] for t in teams) <= 1)

# Wenn ein Spieler von einem Team in einer Woche ausgewählt wird, muss er für die gesamte Saison ausgewählt sein
for i in players:
    for t in teams:
        m.addConstr(y[i, t] >= quicksum(x[i, j, t] for j in weeks) / len(weeks))

# Ein Spieler kann nur von einem Team für die gesamte Saison ausgewählt werden
for i in players:
    m.addConstr(quicksum(y[i, t] for t in teams) <= 1)

# Für Manager außer Manager_1, wähle den Spieler mit dem besten verfügbaren Ranking
# Für Manager außer Manager_1, wähle die Spieler basierend auf dem saisonalen Ranking
for t in teams:
    if t != 'Manager_1':
        #m.addConstr(quicksum(y[i, t] * season_ranking[i] for i in players) <= len(players))
        m.addConstr(quicksum(y[i, t] * season_ranking[i] for i in players if i in season_ranking and (i, t) in y) <= len(players))


# Jedes Team kann nur eine bestimmte Anzahl von Spielern pro Saison auswählen
for t in teams:
    m.addConstr(quicksum(y[i, t] for i in players) <= num_players_per_team)

# Draft-Reihenfolge-Nebenbedingungen
# Draft-Reihenfolge-Nebenbedingungen
for r in rounds:
    for t in draft_order[r-1]:
        m.addConstr(z[t, r] == 1)

# Problem lösen
m.optimize()

# Ausgabe der Ergebnisse
for v in m.getVars():
    if v.x > 0:
        print('%s : %g' % (v.varName, v.x))

print('Total points: %g' % m.objVal)

# Extrahieren der ausgewählten Spieler für jedes Team und jede Woche
selected_players = [(i, j, t) for i in players for j in weeks for t in teams if x[i, j, t].x == 1]

# Umwandlung der Liste in ein DataFrame für eine bessere Darstellung
team_df = pd.DataFrame(selected_players, columns=['Player ID', 'Week', 'Team'])

team_df

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.fantasypros.com/nfl/adp/ppr-overall.php?year=2019'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', attrs={'id': 'data'})

headers = [header.text for header in table.find_all('th')]
headers.remove('Sleeper') # 'Sleeper' Spalte wird entfernt, da sie leer ist

rows = table.find_all('tr')
table_data = []

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    row_data = [ele for ele in cols if ele]
    
    if len(row_data) == len(headers):  # Nur Zeilen hinzufügen, die die richtige Anzahl von Spalten haben
        table_data.append(row_data)

df = pd.DataFrame(table_data, columns=headers)

# Hinzufügen der 'Sleeper' Spalte mit NaN Werten
df['Sleeper'] = pd.np.nan 

# Liste aller Team-Kürzel
team_abbrs = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 
              'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 
              'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 
              'SEA', 'SF', 'TB', 'TEN', 'WAS', 'II', 'Jr.' ,'DST'] 
def extract_info(row):
    player = row
    team = None
    bye = None

    # Extrahieren Sie die Bye-Week, wenn sie vorhanden ist
    if '(' in row and ')' in row:
        bye = row[row.index('(')+1:row.index(')')]
        player = row[:row.index('(')].strip()

    # Prüfen Sie, ob das letzte Wort im Spieler-Feld ein Team-Kürzel ist
    last_word = player.split(' ')[-1]
    if last_word in team_abbrs:
        team = last_word
        player = ' '.join(player.split(' ')[:-1])
        
    # Prüfen Sie, ob das letzte Wort im Spieler-Feld "Jr." oder "II" ist
    last_word = player.split(' ')[-1]
    if last_word in ["Jr.", "II"]:
        player = ' '.join(player.split(' ')[:-1])

    return player, team, bye

# Anwendung der Funktion auf jede Zeile in der "Player Team (Bye)" Spalte
df[['Player', 'Team', 'Bye']] = df['Player Team (Bye)'].apply(extract_info).apply(pd.Series)

# Löschen der ursprünglichen Spalte "Player Team (Bye)"
df = df.drop(columns=['Player Team (Bye)'])

# Zusammenführen der beiden DataFrames auf der Basis unterschiedlicher Spaltennamen
df = pd.merge(df, player_data[['player_display_name', 'player_id']], left_on='Player', right_on='player_display_name', how='left')

# Löschen der zusätzlichen Spalte "player_display_name"
df = df.drop(columns=['player_display_name'])


# Entfernen von Duplikaten und Beibehalten des ersten Auftreffens jeder player_id
df = df.drop_duplicates(subset='player_id', keep='first')

print(df)

